In [1]:
!pip install vadersentiment
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
import gensim
from nltk.stem import WordNetLemmatizer, SnowballStemmer
np.random.seed(10)
import nltk
from bertopic import BERTopic
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
import seaborn as sns
import plotly.express as px
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
def clean_text(text):
    text = text.lower() # lowercase everything
    text = text.encode('ascii', 'ignore').decode()  # remove unicode characters
    text = re.sub(r'https*\S+', ' ', text) # remove links
    text = re.sub(r'http*\S+', ' ', text)
    # cleaning up text
    text = re.sub(r'\'\w+', '', text) 
    text = re.sub(r'\w*\d+\w*', '', text)
    text = re.sub(r'\s{2,}', ' ', text)
    text = re.sub(r'\s[^\w\s]\s', '', text)
    text = re.sub("[\(\[].*?[\)\]]", "", text)
    text = re.sub('[^a-zA-Z0-9 \.]', '', text)
    return text

In [4]:
def lemmatize(text):
  return WordNetLemmatizer().lemmatize(text, pos='v')
def preprocess(text):
    if type(text)!=str:
      return ''
    result = []
    text = text.replace('[removed]', '').replace('[deleted]', '')
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 2:
            result.append(lemmatize(token))
    return ' '.join(result)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import pandas as pd
file_name = "/content/drive/MyDrive/raw_reddit_news_posts_comments2.csv"
df = pd.read_csv(file_name)
df.head()

,Unnamed: 0,id,subreddit,author,created_utc,domain,url,title,num_comments,selftext,score,comments,url_content
0,0,eib002,worldnews,DoremusJessup,2020-01-01 00:00:08,reuters.com,https://www.reuters.com/article/us-india-citiz...,Thousands of Indians ushered in the New Year b...,17,NaN,1,[deleted]...Because most Indians are not Musli...,NEW DELHI (Reuters) - Thousands of Indians ush...
1,1,eib9ut,worldnews,skippy_smooth,2020-01-01 00:21:23,bgr.com,https://bgr.com/2019/12/31/china-pneumonia-sic...,A scary unidentified virus is spreading in China,70,NaN,1,"[removed]...Ah, a fellow time traveler....Well...","Dozens of residents of Wuhan, the capital city..."
2,2,eibebq,worldnews,vannybros,2020-01-01 00:31:16,cnn.com,https://www.cnn.com/2019/12/31/politics/north-...,Kim Jong Un warns hostile US policy means ther...,10,NaN,1,This guy...Trump will comfort Kim....Being tha...,Washington CNN —\n\nNorth Korean Leader Kim Jo...
3,3,eibo17,worldnews,OldFashionedJizz,2020-01-01 00:53:25,theguardian.com,https://www.theguardian.com/world/2019/dec/31/...,"North Korean leader to end missile test ban, c...",7,NaN,1,[removed]...That's a choice of course. We can ...,Kim Jong-un has signalled that North Korea wil...
4,4,eibo8f,worldnews,didnt_see_that_tree,2020-01-01 00:53:55,independent.co.uk,https://www.independent.co.uk/life-style/gadge...,Snapchat has stopped working for users around ...,8,NaN,1,Oh The Humanity ! ! !...OMG....what will teena...,For free real time breaking news alerts sent s...


In [7]:
print(df.shape)
df.fillna("", inplace=True)
len(df[df.id==""])

(136562, 13)


0

In [8]:
# df[df['selftext'].isna()] = ""
df["title"] = df["title"].astype(str) 
df["selftext"] = df["selftext"].astype(str)
df["title"] = df["title"] + " " + df["selftext"]

In [9]:
df['cleaned_title'] = df.title.apply(preprocess)
titles = df["cleaned_title"].astype(str).to_list()

In [10]:
topic_model = BERTopic(min_topic_size=100, n_gram_range=(1,3), nr_topics='auto', verbose=True)
topics, _ = topic_model.fit_transform(titles)

Batches:   0%|          | 0/4268 [00:00<?, ?it/s]

2022-12-03 22:11:51,192 - BERTopic - Transformed documents to Embeddings
2022-12-03 22:15:06,799 - BERTopic - Reduced dimensionality
2022-12-03 22:15:29,305 - BERTopic - Clustered reduced embeddings
2022-12-03 22:16:00,982 - BERTopic - Reduced number of topics from 190 to 47


In [11]:
topic_model.save("/content/drive/MyDrive/topic_news_model")

/usr/local/lib/python3.8/dist-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [12]:
barchart = topic_model.visualize_barchart(top_n_topics=10, n_words=10, height=400)
barchart

In [13]:
barchart.write_html("/content/drive/MyDrive/news_word_freq.html")

In [14]:
intertopic_dist = topic_model.visualize_topics()
intertopic_dist

In [15]:
intertopic_dist.write_html("/content/drive/MyDrive/news_intertopic_distance.html")

In [16]:
freq = topic_model.get_topic_info()
freq.head(10)

,Topic,Count,Name
0,-1,58310,-1_say_covid_ukraine_china
1,0,64932,0_china_coronavirus_say_covid
2,1,1140,1_mask_face mask_wear_face
3,2,1082,2_space_earth_nasa_moon
4,3,828,3_climate_climate change_change_emissions
5,4,699,4_pope_vatican_francis_pope francis
6,5,662,5_navalny_alexei_alexei navalny_poison
7,6,605,6_zelensky_zelenskyy_ukraine_volodymyr
8,7,540,7_amazon_deforestation_rainforest_tree
9,8,445,8_australia_bushfires_wildfires_bushfire


In [17]:
topic_nr = freq.iloc[3]["Topic"]  # We select a frequent topic
topic_model.get_topic(topic_nr)

[('space', 0.058229225499376314),
 ('earth', 0.03388060342632617),
 ('nasa', 0.028047603602742263),
 ('moon', 0.02690213453811982),
 ('space station', 0.02523557856160365),
 ('asteroid', 0.02425323528654739),
 ('station', 0.022107480316083027),
 ('spacex', 0.02142150138461926),
 ('rocket', 0.016769424211301537),
 ('launch', 0.015619939514922642)]

In [18]:
freq["word_strength"] = freq.Topic.apply(topic_model.get_topic)
freq.head()

,Topic,Count,Name,word_strength
0,-1,58310,-1_say_covid_ukraine_china,"[(say, 0.006664784405644968), (covid, 0.006658..."
1,0,64932,0_china_coronavirus_say_covid,"[(china, 0.008668904388153988), (coronavirus, ..."
2,1,1140,1_mask_face mask_wear_face,"[(mask, 0.1258858160698568), (face mask, 0.045..."
3,2,1082,2_space_earth_nasa_moon,"[(space, 0.058229225499376314), (earth, 0.0338..."
4,3,828,3_climate_climate change_change_emissions,"[(climate, 0.09209322590849058), (climate chan..."


In [19]:
freq.to_csv("/content/drive/MyDrive/news_topic_frequency.csv", index=False)

In [20]:
df['created_utc'] = pd.to_datetime(df['created_utc'])
df["created_dt_month"] = df['created_utc'].dt.strftime('%m-%Y')
titles = df["cleaned_title"].astype(str).to_list()
dates = df['created_utc']
topics_over_time = topic_model.topics_over_time(docs=titles,  
                                                # topics=topics, 
                                                timestamps=dates, 
                                                global_tuning=True, 
                                                evolution_tuning=True, 
                                                nr_bins=30)

27it [09:31, 21.16s/it]


In [21]:
df['created_dt_month'].value_counts()

03-2022    11184
02-2022     8789
03-2020     7632
04-2020     6995
01-2020     6915
05-2021     5895
07-2020     5706
08-2021     5423
06-2020     5376
05-2020     5324
02-2020     5193
06-2021     5026
12-2020     4884
09-2021     4746
07-2021     4695
08-2020     4675
11-2020     4576
01-2021     4545
04-2021     4458
10-2021     4254
10-2020     4251
09-2020     3961
11-2021     3832
02-2021     3789
03-2021     2625
12-2021     1677
04-2022       70
01-2022       60
09-2022        6
Name: created_dt_month, dtype: int64

In [22]:
tot = topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)
tot

In [23]:
tot.write_html("/content/drive/MyDrive/news_topics_over_time.html")

In [24]:
topics_over_time.to_csv("/content/drive/MyDrive/topics_over_time_news.csv", index=False)
representative_docs = topic_model.get_representative_docs()

docs_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in representative_docs.items() ])).T
docs_df.index.name = "topic_nr"
docs_df.to_csv("/content/drive/MyDrive/representative_docs_news.csv")
docs_df.head()

,0,1,2,3,4,5,6,7,8,9,...,413,414,415,416,417,418,419,420,421,422
topic_nr,,,,,,,,,,,,,,,,,,,,,
0,india modi lash protest farmers,climate activist arrest india farmers protest ...,cold rain india farmers press stand modi,investigator claim china refuse hand key covid...,china blast overreaction virus spread fear,covid disinformation spread russia china experts,msnbc katy tur delete tweet apologize report k...,north korea say see improvement relations main...,kim kardashian kourtney kardashian physical al...,facebook commitment free speech piss people zu...,...,ukraine offer usd asylum country russian solid...,ukrainian pilot arrive poland pick donate figh...,bulgaria poland slovakia donate fighter jet uk...,pentagon reject nato nations provide jet ukraine,putin mull recognize separatist eastern ukrain...,russia recognise separatist area independent,ukrainian mps vote recognize independence russ...,worldnews live thread russian invasion ukraine...,worldnews live thread russian invasion ukraine...,worldnews live thread russian invasion ukraine...
14,face backlash tiktok rat drop play store,amazon say ban tiktok employee phone,tiktok collect children personal information p...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,new zealand send troop help australian bushfir...,megafire measure million acres form australia ...,disturb video firestorm australia show quickly,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,pope apologize slap hand woman try yank,children victims sex abuse france catholic chu...,pedophiles work french catholic church indepen...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,war plastic strand oil industry investment,global treaty end production virgin plastic,malaysia send ship containers plastic waste we...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df['topic'] = df['cleaned_title'].apply(lambda x: topic_model.find_topics(x)[0][0])
df.head()

,Unnamed: 0,id,subreddit,author,created_utc,domain,url,title,num_comments,selftext,score,comments,url_content,cleaned_title,created_dt_month,topic
0,0,eib002,worldnews,DoremusJessup,2020-01-01 00:00:08,reuters.com,https://www.reuters.com/article/us-india-citiz...,Thousands of Indians ushered in the New Year b...,17,,1,[deleted]...Because most Indians are not Musli...,NEW DELHI (Reuters) - Thousands of Indians ush...,thousands indians usher new year demonstrate c...,01-2020,45
1,1,eib9ut,worldnews,skippy_smooth,2020-01-01 00:21:23,bgr.com,https://bgr.com/2019/12/31/china-pneumonia-sic...,A scary unidentified virus is spreading in China,70,,1,"[removed]...Ah, a fellow time traveler....Well...","Dozens of residents of Wuhan, the capital city...",scary unidentified virus spread china,01-2020,0
2,2,eibebq,worldnews,vannybros,2020-01-01 00:31:16,cnn.com,https://www.cnn.com/2019/12/31/politics/north-...,Kim Jong Un warns hostile US policy means ther...,10,,1,This guy...Trump will comfort Kim....Being tha...,Washington CNN —\n\nNorth Korean Leader Kim Jo...,kim jong warn hostile policy mean korean penin...,01-2020,0
3,3,eibo17,worldnews,OldFashionedJizz,2020-01-01 00:53:25,theguardian.com,https://www.theguardian.com/world/2019/dec/31/...,"North Korean leader to end missile test ban, c...",7,,1,[removed]...That's a choice of course. We can ...,Kim Jong-un has signalled that North Korea wil...,north korean leader end missile test ban claim...,01-2020,0
4,4,eibo8f,worldnews,didnt_see_that_tree,2020-01-01 00:53:55,independent.co.uk,https://www.independent.co.uk/life-style/gadge...,Snapchat has stopped working for users around ...,8,,1,Oh The Humanity ! ! !...OMG....what will teena...,For free real time breaking news alerts sent s...,snapchat stop work users world tens thousands ...,01-2020,14


In [26]:
df['cleaned_comments'] = df['comments'].map(preprocess)
df.to_csv("/content/drive/MyDrive/reddit_news_data_with_topics.csv", index=False)
df.head()

,Unnamed: 0,id,subreddit,author,created_utc,domain,url,title,num_comments,selftext,score,comments,url_content,cleaned_title,created_dt_month,topic,cleaned_comments
0,0,eib002,worldnews,DoremusJessup,2020-01-01 00:00:08,reuters.com,https://www.reuters.com/article/us-india-citiz...,Thousands of Indians ushered in the New Year b...,17,,1,[deleted]...Because most Indians are not Musli...,NEW DELHI (Reuters) - Thousands of Indians ush...,thousands indians usher new year demonstrate c...,01-2020,45,indians muslims million muslim feel real citiz...
1,1,eib9ut,worldnews,skippy_smooth,2020-01-01 00:21:23,bgr.com,https://bgr.com/2019/12/31/china-pneumonia-sic...,A scary unidentified virus is spreading in China,70,,1,"[removed]...Ah, a fellow time traveler....Well...","Dozens of residents of Wuhan, the capital city...",scary unidentified virus spread china,01-2020,0,fellow time traveler good turn interest gamepl...
2,2,eibebq,worldnews,vannybros,2020-01-01 00:31:16,cnn.com,https://www.cnn.com/2019/12/31/politics/north-...,Kim Jong Un warns hostile US policy means ther...,10,,1,This guy...Trump will comfort Kim....Being tha...,Washington CNN —\n\nNorth Korean Leader Kim Jo...,kim jong warn hostile policy mean korean penin...,01-2020,0,guy trump comfort kim intentions nuke meaningl...
3,3,eibo17,worldnews,OldFashionedJizz,2020-01-01 00:53:25,theguardian.com,https://www.theguardian.com/world/2019/dec/31/...,"North Korean leader to end missile test ban, c...",7,,1,[removed]...That's a choice of course. We can ...,Kim Jong-un has signalled that North Korea wil...,north korean leader end missile test ban claim...,01-2020,0,choice course agree big problem north korea pu...
4,4,eibo8f,worldnews,didnt_see_that_tree,2020-01-01 00:53:55,independent.co.uk,https://www.independent.co.uk/life-style/gadge...,Snapchat has stopped working for users around ...,8,,1,Oh The Humanity ! ! !...OMG....what will teena...,For free real time breaking news alerts sent s...,snapchat stop work users world tens thousands ...,01-2020,14,humanity omg teenagers world snap corner carpe...


In [27]:
sentiment_analyzer = SentimentIntensityAnalyzer()
df["comment_sentiment"] = df['cleaned_comments'].apply(lambda x: sentiment_analyzer.polarity_scores(x)["compound"])

In [28]:
df['created_utc'] = pd.to_datetime(df['created_utc'])
df["created_dt_month"] = df['created_utc'].dt.strftime('%m-%Y')

In [29]:
sentiment_df = df[["created_dt_month", "topic", "comment_sentiment"]].groupby(["created_dt_month", "topic"]).mean()
sentiment_df.head()

comment_sentiment
created_dt_month topic                   
01-2020          -1             -0.105845
                  0             -0.217111
                  1             -0.000613
                  2              0.378485
                  3              0.117338

In [30]:
sentiment_df = sentiment_df.reset_index()

In [31]:
map_topic = freq[["Topic", "Name"]]
map_topic.columns = ['topic', 'name']
merged_df = sentiment_df.merge(map_topic, on=['topic'])

In [32]:
merged_df["created_dt_month"] = pd.to_datetime(merged_df["created_dt_month"])
merged_df = merged_df.sort_values("created_dt_month")
merged_df = merged_df[merged_df['topic']!=-1]
merged_df.head()

,created_dt_month,topic,comment_sentiment,name
1129,2020-01-01,41,-0.146689,41_toilet_toilet paper_paper_hand
378,2020-01-01,13,0.258742,13_queen_prince_elizabeth_queen elizabeth
1103,2020-01-01,40,0.238156,40_wheat_food_ukraine_war
620,2020-01-01,22,0.106357,22_greta_thunberg_greta thunberg_climate
189,2020-01-01,6,-0.424497,6_zelensky_zelenskyy_ukraine_volodymyr


In [33]:
plot_df = merged_df[merged_df['topic'].isin([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])]
# plot_df["created_dt_month"] = pd.to_datetime(plot_df["created_dt_month"])
fig = px.line(plot_df, x="created_dt_month", y="comment_sentiment", color="name")
fig.show()
# sns.lineplot(data=plot_df, x="created_dt_month", y="comment_sentiment", hue="name")

In [34]:
fig.write_html("/content/drive/MyDrive/news_comments_sentiments.html")

In [35]:
merged_df.to_csv("/content/drive/MyDrive/news_comment_sentiment.csv", index=False)

In [36]:
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
docs = titles
# topic_model = BERTopic(verbose=True, n_gram_range=(1, 3),nr_topics='auto',min_topic_size=100)
# topics, _ = topic_model.fit_transform(docs)

# Preprocess Documents
documents = pd.DataFrame({"Document": docs,
                          "ID": range(len(docs)),
                          "Topic": topics})
documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
cleaned_docs = topic_model._preprocess_text(documents_per_topic.Document.values)

# Extract vectorizer and analyzer from BERTopic
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()

# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
               for topic in range(len(set(topics))-1)]

# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_v')
coherence = coherence_model.get_coherence()
print(coherence)

0.4175703779654064
